In [1]:
import numpy as np
import pandas as pd
import time
import logging

from imblearn.combine import SMOTEENN
from catboost import CatBoostClassifier, EShapCalcType, EFeaturesSelectionAlgorithm, Pool, EFstrType
from catboost.utils import get_roc_curve, select_threshold
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import roc_auc_score, average_precision_score, precision_score, f1_score, roc_curve, precision_recall_curve, recall_score, auc
from sklearn.preprocessing import LabelBinarizer
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import PrecisionRecallDisplay
from yellowbrick.classifier import PrecisionRecallCurve

In [2]:
logger = logging.getLogger(__name__)

In [3]:
X, y = make_classification(n_samples=6000, n_clusters_per_class=10, n_classes=5, n_informative=500, n_features=800)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y)
X_train, y_train

(array([[-1.60531288e-01, -8.36387200e-02,  2.11242302e-03, ...,
         -2.12397138e+01,  2.04020476e+01, -5.57614427e+00],
        [-2.31393561e-01, -1.21270918e-02, -1.22404901e-01, ...,
         -3.19893897e+00,  6.86940043e+00,  1.16876144e+00],
        [ 1.68694302e+00,  1.22294827e+00, -3.38487634e-01, ...,
          8.04861209e+00,  1.36455899e+00, -3.49518066e+00],
        ...,
        [ 1.33440561e+00,  1.46389536e+00, -4.50622645e-01, ...,
         -1.88384790e+01, -3.76788205e+00,  1.23012704e+01],
        [ 5.49243615e-01,  8.79108233e-03, -1.92823409e-02, ...,
         -1.36368169e+01, -5.87378897e+00,  4.91993953e+00],
        [ 4.02219909e-01, -1.01930731e+00,  5.22493242e-02, ...,
          6.35001319e+00, -5.66685952e+00, -1.89930518e+01]]),
 array([1, 0, 1, ..., 4, 2, 2]))

In [13]:
# Address class imbalance
resampler = SMOTE(sampling_strategy="not majority")
X_train_res, y_train_res = resampler.fit_resample(X_train, y_train)

# Create Pool objs for catboost
train_data = Pool(data=X_train, label=y_train)
eval_data = Pool(data=X_test, label=y_test)

clf = CatBoostClassifier(
    iterations=1000, loss_function="MultiClass", allow_writing_files=False,
    early_stopping_rounds=40, task_type="CPU", verbose=True)
# clf.fit(X_train, y=y_train, eval_set=eval_data)

In [19]:
np.arange(len(X_train[0, :]))

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [20]:
all_feature_idx = np.arange(len(X_train[0, :]))

summary = clf.select_features(
    X=train_data, eval_set=eval_data, num_features_to_select=500,
    features_for_select=all_feature_idx, steps=2,
    algorithm=EFeaturesSelectionAlgorithm.RecursiveByLossFunctionChange,
    shap_calc_type=EShapCalcType.Approximate, train_final_model=True, verbose=False)

Learning rate set to 0.110375
Step #1 out of 2
Stopped by overfitting detector  (40 iterations wait)

bestTest = 1.60994504
bestIteration = 1

Shrink model to first 2 iterations.
Feature #282 eliminated
Feature #235 eliminated
Feature #17 eliminated
Feature #155 eliminated
Feature #729 eliminated
Feature #295 eliminated
Feature #599 eliminated
Feature #508 eliminated
Feature #191 eliminated
Feature #9 eliminated
Feature #10 eliminated
Feature #11 eliminated
Feature #12 eliminated
Feature #13 eliminated
Feature #14 eliminated
Feature #15 eliminated
Feature #16 eliminated
Feature #8 eliminated
Feature #18 eliminated
Feature #19 eliminated
Feature #20 eliminated
Feature #21 eliminated
Feature #22 eliminated
Feature #23 eliminated
Feature #24 eliminated
Feature #25 eliminated
Feature #26 eliminated
Feature #27 eliminated
Feature #28 eliminated
Feature #29 eliminated
Feature #30 eliminated
Feature #31 eliminated
Feature #32 eliminated
Feature #33 eliminated
Feature #34 eliminated
Feature #3

In [32]:
summary.keys()

dict_keys(['selected_features', 'eliminated_features_names', 'loss_graph', 'eliminated_features', 'selected_features_names'])

In [37]:
selected = pd.DataFrame(summary["selected_features_names"], columns=["feature_name"])
selected["selected"] = True
eliminated = pd.DataFrame(summary["eliminated_features_names"], columns=["feature_name"])
eliminated["selected"] = False
feature_df = pd.concat([selected, eliminated], ignore_index=True)
feature_df

,feature_name,selected
0,,True
1,,True
2,,True
3,,True
4,,True
...,...,...
795,,False
796,,False
797,,False
798,,False


In [38]:
clf.get_feature_importance(data=train_data,
                       reference_data=eval_data,
                       type=EFstrType.ShapValues,
                       prettified=True,
                       thread_count=-1,
                       verbose=True)

Used MultiClass metric for fstr calculation
Processing trees...
Processing documents...
128/3000 documents processed	passed time: 394ms	remaining time: 8.84s
256/3000 documents processed	passed time: 694ms	remaining time: 7.44s
384/3000 documents processed	passed time: 994ms	remaining time: 6.77s
512/3000 documents processed	passed time: 1.3s	remaining time: 6.3s
640/3000 documents processed	passed time: 1.6s	remaining time: 5.91s
768/3000 documents processed	passed time: 1.91s	remaining time: 5.55s
896/3000 documents processed	passed time: 2.21s	remaining time: 5.2s
1024/3000 documents processed	passed time: 2.52s	remaining time: 4.86s
1152/3000 documents processed	passed time: 2.82s	remaining time: 4.53s
1280/3000 documents processed	passed time: 3.13s	remaining time: 4.2s
1408/3000 documents processed	passed time: 3.43s	remaining time: 3.88s
1536/3000 documents processed	passed time: 3.73s	remaining time: 3.56s
1664/3000 documents processed	passed time: 4.04s	remaining time: 3.24s
1

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00, -8.23760626e-06],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  1.73510720e-03],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00, -1.04013122e-03],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  9.99377568e-04],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00, -1.68611595e-03]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00, -8.23760626e-06],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  1.73510720e-03],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00, -1.04013

In [ ]:
# Model performance metrics
encoder = LabelBinarizer()
encoder.fit(y_train)
y_test_enc = encoder.transform(y_test)
y_pred_proba = clf.predict_proba(X_test)

In [ ]:
# Model performance metrics
for i, c in enumerate(encoder.classes_):
    # AUC per class
    roc_auc = roc_auc_score(y_test_enc[:, i], y_pred_proba[:, i], average=None)
    # dk.data['extra_returns_per_train'][f"roc_auc_{c}"] = roc_auc
    print(f"ROC AUC score: {roc_auc}")

    auprc = average_precision_score(y_test_enc[:, i], y_pred_proba[:, i])
    print(f"PR AUC score: {auprc}")

In [ ]:
imp = clf.get_feature_importance(data=eval_data, type=EFstrType.LossFunctionChange, prettified=True, shap_calc_type="Approximate")
imp

In [ ]:
imp.sort_values(by="Importances", ascending=False)

In [ ]:
for i, c in enumerate(encoder.classes_):
    roc_auc = roc_auc_score(y_test_enc[:, i], y_pred_proba[:, i])
    print(c, roc_auc)
    
    fpr, tpr, t1 = roc_curve(y_test_enc[:, i], y_pred_proba[:, i])
    print(fpr, tpr, t1)
    
    precision, recall, t2 = precision_recall_curve(y_test_enc[:, i], y_pred_proba[:, i])
    print(precision, recall, t2)
    
    

In [ ]:
t2

In [ ]:
fpr

In [ ]:
def get_threshold(search_list, lookup_list, search_threshold):
    comp = search_list > search_threshold
    if comp.sum() == 0:
        return 1
    idx = np.argmax(comp)
    return lookup_list[idx]

In [ ]:
precision > 1.9

In [ ]:
get_threshold(precision, t2, -.9)

In [ ]:
precision, t2

In [ ]:
??_select_threshold

In [ ]:
precision_recall_curve

import 